### Data ###

In [1]:
import xray

In [2]:
data = xray.open_dataset('tracks.20130125.nc', decode_cf=False)

In [3]:
print data

<xray.Dataset>
Dimensions:  (Nobs: 2590938)
Coordinates:
  * Nobs     (Nobs) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
Data variables:
    track    (Nobs) int32 1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 5 5 5 5 6 6 6 6 7 ...
    n        (Nobs) int32 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 ...
    j1       (Nobs) int32 2448910 2448917 2448924 2448931 2448910 2448917 ...
    cyc      (Nobs) int32 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...
    lon      (Nobs) float32 636.74 635.99 635.82 635.845 560.642 561.447 ...
    lat      (Nobs) float32 -65.0011 -65.1376 -65.2457 -65.2933 -62.5424 ...
    A        (Nobs) float32 1.24564 2.64604 3.40119 1.45836 2.16156 4.87579 ...
    L        (Nobs) float32 44.464 36.7024 45.4221 43.022 63.0697 51.1345 ...
    U        (Nobs) float32 5.97993 8.17373 8.28628 5.20364 4.81408 6.59872 ...
Attributes:
    title: Mesoscale Eddies in Altimeter Observations of SSH
    institution: Oregon State University/CEOAS
    file_name: t

### Multi-Index ###

In [4]:
import numpy as np
import pandas as pd

In [5]:
ds = data.to_dataframe()

In [6]:
ds.tail(8)

,track,n,j1,cyc,lon,lat,A,L,U
Nobs,,,,,,,,,
2590930,215183,1,2456001,1,307.384003,59.680500,1.78922,44.618099,5.37469
2590931,215183,2,2456008,1,307.834015,59.388100,3.08864,41.930099,10.53940
2590932,215183,3,2456015,1,307.615997,59.352699,4.55595,48.856499,13.26870
2590933,215183,4,2456022,1,307.500000,59.202999,5.27399,58.372101,14.44380
2590934,215184,1,2456001,1,336.242004,62.275398,3.06445,54.620499,7.21748
2590935,215184,2,2456008,1,336.033997,62.011700,2.03734,53.794102,5.87109
2590936,215184,3,2456015,1,335.938995,62.359200,1.75523,30.240000,6.72342
2590937,215184,4,2456022,1,335.752991,62.145901,1.23239,37.190102,6.36023


In [7]:
# Dimension Setting

# All
# M = max(ds.track)
# N = len(ds.n)

# TEST
M = 1000
N = 17687
ds = ds.head(N)

In [8]:
ds.tail(4)

,track,n,j1,cyc,lon,lat,A,L,U
Nobs,,,,,,,,,
17683,1000,5,2448945,-1,581.109985,46.809700,1.90537,66.215698,5.11371
17684,1000,6,2448952,-1,581.013977,46.808102,2.50892,68.702003,5.97126
17685,1000,7,2448959,-1,580.973022,46.610298,2.08100,52.068699,5.61743
17686,1000,8,2448966,-1,580.992004,46.574100,1.48384,58.216400,5.03651


In [9]:
track = np.asarray(ds.track)
n = np.asarray(ds.n)

arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['Eddy ID', 'Point ID'])

In [10]:
j1 = np.asarray(ds.j1)
cyc = np.asarray(ds.cyc)
lon = np.asarray(ds.lon)
lat = np.asarray(ds.lat)
A = np.asarray(ds.A)
L = np.asarray(ds.L)
U = np.asarray(ds.U)

info = np.zeros((N, 7))
for i in range(0, N):
    info[i][0] = j1[i]
    info[i][1] = cyc[i]
    info[i][2] = lon[i]
    info[i][3] = lat[i]
    info[i][4] = A[i]
    info[i][5] = L[i]
    info[i][6] = U[i]

In [11]:
df = pd.DataFrame(info, index=multi_index)
df.columns = ['j1', 'cyc', 'lon','lat', 'A', 'L', 'U']

### Time ###

In [12]:
import datetime, jdcal

In [13]:
df.tail(4)

j1  cyc         lon        lat        A          L  \
Eddy ID Point ID                                                            
1000    5         2448945   -1  581.109985  46.809700  1.90537  66.215698   
        6         2448952   -1  581.013977  46.808102  2.50892  68.702003   
        7         2448959   -1  580.973022  46.610298  2.08100  52.068699   
        8         2448966   -1  580.992004  46.574100  1.48384  58.216400   

                        U  
Eddy ID Point ID           
1000    5         5.11371  
        6         5.97126  
        7         5.61743  
        8         5.03651

In [14]:
def jday_to_datetime(jday, refday=0):
    y, m, d, f = jdcal.jd2gcal(jday, refday)
    h = int(f*24)
    return pd.to_datetime(datetime.datetime(y, m, d, h))

In [15]:
pd_date = df.j1.apply(jday_to_datetime)

In [16]:
df.j1 = pd_date
df = df.rename(columns = {'j1':'date'})

### Longitude ###

In [17]:
df.tail(4)

date  cyc         lon        lat        A  \
Eddy ID Point ID                                                            
1000    5        1992-11-18 12:00:00   -1  581.109985  46.809700  1.90537   
        6        1992-11-25 12:00:00   -1  581.013977  46.808102  2.50892   
        7        1992-12-02 12:00:00   -1  580.973022  46.610298  2.08100   
        8        1992-12-09 12:00:00   -1  580.992004  46.574100  1.48384   

                          L        U  
Eddy ID Point ID                      
1000    5         66.215698  5.11371  
        6         68.702003  5.97126  
        7         52.068699  5.61743  
        8         58.216400  5.03651

In [18]:
lon_fix_01 = df.where(df.lon <= 540).lon - 360
lon_fix_02 = df.where(df.lon > 540).lon - 720

lon_fix = lon_fix_01
lon_fix = lon_fix.fillna(lon_fix_02)
df.lon = lon_fix

In [19]:
df.tail(4)

date  cyc         lon        lat        A  \
Eddy ID Point ID                                                            
1000    5        1992-11-18 12:00:00   -1 -138.890015  46.809700  1.90537   
        6        1992-11-25 12:00:00   -1 -138.986023  46.808102  2.50892   
        7        1992-12-02 12:00:00   -1 -139.026978  46.610298  2.08100   
        8        1992-12-09 12:00:00   -1 -139.007996  46.574100  1.48384   

                          L        U  
Eddy ID Point ID                      
1000    5         66.215698  5.11371  
        6         68.702003  5.97126  
        7         52.068699  5.61743  
        8         58.216400  5.03651

### Point ###

In [20]:
ds.tail(4)

,track,n,j1,cyc,lon,lat,A,L,U
Nobs,,,,,,,,,
17683,1000,5,2448945,-1,581.109985,46.809700,1.90537,66.215698,5.11371
17684,1000,6,2448952,-1,581.013977,46.808102,2.50892,68.702003,5.97126
17685,1000,7,2448959,-1,580.973022,46.610298,2.08100,52.068699,5.61743
17686,1000,8,2448966,-1,580.992004,46.574100,1.48384,58.216400,5.03651


In [21]:
count = ds.track
count = count.value_counts(normalize=False, sort=True, ascending=True, bins=None, dropna=False).reindex(range(1, M+1))
count = pd.DataFrame(count)
count.columns = ['point']

In [22]:
count.tail(4)

,point
997,8
998,8
999,8
1000,8


In [23]:
lon = np.asarray(df.lon)
lat = np.asarray(df.lat)
point = np.asarray(count.point)

In [24]:
lon_o = np.zeros((M, 1))
lat_o = np.zeros((M, 1))

lon_t = np.zeros((M, 1))
lat_t = np.zeros((M, 1))

In [25]:
i = 0
c = 0

while i < N:
    c = int(c) + 1
    lon_o[c-1] = lon[i]
    lat_o[c-1] = lat[i]
    i = i + int(point[c-1])
    lon_t[c-1] = lon[i-1]
    lat_t[c-1] = lat[i-1]

In [26]:
lon_o = pd.DataFrame(lon_o)
lat_o = pd.DataFrame(lat_o)

lon_t = pd.DataFrame(lon_t)
lat_t = pd.DataFrame(lat_t)

In [27]:
index = np.zeros(M)

In [28]:
for i in range(0, M):
    index[i] = i+1

In [29]:
index = pd.DataFrame(index)

In [30]:
# Start Centers
sc = pd.concat([index, lon_o, lat_o], axis=1)
sc.columns = ['Eddy ID','lon', 'lat']
sc = sc.set_index('Eddy ID')

# End Centers
ec = pd.concat([index, lon_t, lat_t], axis=1)
ec.columns = ['Eddy ID','lon', 'lat']
ec = ec.set_index('Eddy ID')

In [31]:
sc.tail(4)

,lon,lat
Eddy ID,,
997,-91.752991,-32.631199
998,-141.437012,-30.702101
999,-20.184998,29.784700
1000,-138.401978,47.121700


In [32]:
ec.tail(4)

,lon,lat
Eddy ID,,
997,-91.273987,-33.806702
998,-141.560974,-29.453501
999,-21.278015,29.068501
1000,-139.007996,46.574100


### LineString ###

In [33]:
linestring = np.zeros((N, 2))

for i in range(0, N):
    linestring[i][0] = lon[i]
    linestring[i][1] = lat[i]

In [34]:
ls = pd.DataFrame(linestring, index=multi_index, columns=['lon', 'lat'])

In [35]:
ls.tail(4)

lon        lat
Eddy ID Point ID                       
1000    5        -138.890015  46.809700
        6        -138.986023  46.808102
        7        -139.026978  46.610298
        8        -139.007996  46.574100

### Polygon ###

In [36]:
from numpy import cos, pi, sin

In [37]:
L = np.asarray(df.L)

In [38]:
L_o = np.zeros(M)
L_t = np.zeros(M)

In [39]:
print N

17687


In [40]:
i = 0
c = 0

while i < N:
    c = int(c) + 1
    L_o[c-1] = L[i]
    i = i + int(point[c-1])
    L_t[c-1] = L[i-1]

In [41]:
L_o = pd.DataFrame(L_o)
L_t = pd.DataFrame(L_t)

In [42]:
# Origin Circles
oc = pd.concat([lon_o, lat_o, L_o], axis=1)
oc.columns = ['lon', 'lat', 'L']

# Termination Circles
tc = pd.concat([lon_t, lat_t, L_t], axis=1)
tc.columns = ['lon', 'lat', 'L']

In [43]:
oc.tail(4)

,lon,lat,L
996,-91.752991,-32.631199,126.373001
997,-141.437012,-30.702101,63.460701
998,-20.184998,29.784700,86.515900
999,-138.401978,47.121700,42.862099


In [44]:
tc.tail(4)

,lon,lat,L
996,-91.273987,-33.806702,49.583099
997,-141.560974,-29.453501,80.245201
998,-21.278015,29.068501,58.712200
999,-139.007996,46.574100,58.216400


In [45]:
# Circle Center Number
C = M

# Circle Arc Number
A = C*17

In [46]:
center = np.zeros(A)
arc = np.zeros(A)

In [47]:
i = 0
j = 1
c = 1

while i < A:
    while c <= 17:
        center[i] = j
        i = i+1
        c = c+1
    j = j+1
    c = 1

In [48]:
center = center.astype(int)

In [49]:
i = 0
j = 1
c = 1

while i < A:
    while c <= 17:
        arc[i] = j
        i = i+1
        j = j+1
        c = c+1
    j = 1
    c = 1

In [50]:
center = center.astype(int)

In [51]:
arrays = [center, arc]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['center', 'arc'])

In [52]:
# Earth Radius in Kilometers
R = 6371

__Origins__

In [53]:
lon = np.asarray(oc.lon)
lat = np.asarray(oc.lat)
L = np.asarray(oc.L)

In [54]:
theta = np.zeros(C)
x = np.zeros(C)
y = np.zeros(C)
r = np.zeros(C)

In [55]:
for i in range(0, C):
    theta[i] = lat[i]*(pi/180)
    r[i] = R*cos(theta[i])
    x[i] = (L[i]/r[i])*(180/pi)
    y[i] = (L[i]/R)*(180/pi)

In [56]:
op_lon = np.zeros(A)
op_lat = np.zeros(A)
op_x = np.zeros(A)
op_y = np.zeros(A)

In [57]:
i = 0
j = 1
c = 0

while i < A:
    while j <= 17:
        op_lon[i] = lon[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [58]:
i = 0
j = 1
c = 0

while i < A:
    while j <= 17:
        op_lat[i] = lat[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [59]:
i = 0
j = 1
c = 0

while i < A:
    while j <= 17:
        op_x[i] = x[c]*cos((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [60]:
i = 0
j = 1
c = 0

while i < A:
    while j <= 17:
        op_y[i] = y[c]*sin((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [61]:
op = np.zeros((A, 2))

In [62]:
for i in range(0, A):
    op[i][0] = op_lon[i]+op_x[i]
    op[i][1] = op_lat[i]+op_y[i]

In [63]:
# Origin Polygons
op = pd.DataFrame(op, index=multi_index)
op.columns = ['lon','lat']

In [64]:
op.tail(4)

lon        lat
center arc                       
1000   14  -138.185189  46.765574
       15  -138.001405  46.849133
       16  -137.878604  46.974188
       17  -137.835482  47.121700

__Terminations__

In [65]:
lon = np.asarray(tc.lon)
lat = np.asarray(tc.lat)
L = np.asarray(tc.L)

In [66]:
theta = np.zeros(C)
r = np.zeros(C)
x = np.zeros(C)
y = np.zeros(C)

In [67]:
for i in range(0, C):
    theta[i] = lat[i]*(pi/180)
    r[i] = R*cos(theta[i])
    x[i] = (L[i]/r[i])*(180/pi)
    y[i] = (L[i]/R)*(180/pi)

In [68]:
tp_lon = np.zeros(A)
tp_lat = np.zeros(A)
tp_x = np.zeros(A)
tp_y = np.zeros(A)

In [69]:
i = 0
j = 1
c = 0

while i < A:
    while j <= 17:
        tp_lon[i] = lon[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [70]:
i = 0
j = 1
c = 0

while i < A:
    while j <= 17:
        tp_lat[i] = lat[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [71]:
i = 0
j = 1
c = 0

while i < A:
    while j <= 17:
        tp_x[i] = x[c]*cos((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [72]:
i = 0
j = 1
c = 0

while i < A:
    while j <= 17:
        tp_y[i] = y[c]*sin((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [73]:
tp = np.zeros((A, 2))

In [74]:
for i in range(0, A):
    tp[i][0] = tp_lon[i]+tp_x[i]
    tp[i][1] = tp_lat[i]+tp_y[i]

In [75]:
# Termination Polygons
tp = pd.DataFrame(tp, index=multi_index)
tp.columns = ['lon','lat']

In [76]:
tp.tail(4)

lon        lat
center arc                       
1000   14  -138.716535  46.090401
       15  -138.469446  46.203893
       16  -138.304346  46.373746
       17  -138.246371  46.574100

### ID ###

In [77]:
eddy_id = pd.DataFrame(index)
eddy_id = pd.concat([index, index], axis=1)
eddy_id.columns = ['Eddy ID', 'eddy id']
eddy_id = eddy_id.set_index('Eddy ID')

In [78]:
eddy_id.tail(4)

,eddy id
Eddy ID,
997,997
998,998
999,999
1000,1000


### Date ###

In [79]:
ds.tail(4)

,track,n,j1,cyc,lon,lat,A,L,U
Nobs,,,,,,,,,
17683,1000,5,2448945,-1,581.109985,46.809700,1.90537,66.215698,5.11371
17684,1000,6,2448952,-1,581.013977,46.808102,2.50892,68.702003,5.97126
17685,1000,7,2448959,-1,580.973022,46.610298,2.08100,52.068699,5.61743
17686,1000,8,2448966,-1,580.992004,46.574100,1.48384,58.216400,5.03651


In [80]:
j1 = np.asarray(ds.j1)

In [81]:
j1_o = np.zeros((M, 1))
j1_t = np.zeros((M, 1))

In [82]:
i = 0
c = 0

while i < N:
    c = int(c) + 1
    j1_o[c-1] = j1[i]
    i = i + int(point[c-1])
    j1_t[c-1] = j1[i-1]

In [83]:
j1_o = pd.DataFrame(j1_o)
j1_t = pd.DataFrame(j1_t)

In [84]:
# Start Julian Dates
sj = pd.concat([index, j1_o], axis=1)
sj.columns = ['Eddy ID','j1']
sj = sj.set_index('Eddy ID')

# End Julian Dates
ej = pd.concat([index, j1_t], axis=1)
ej.columns = ['Eddy ID','j1']
ej = ej.set_index('Eddy ID')

In [85]:
sj.tail(4)

,j1
Eddy ID,
997,2448917
998,2448917
999,2448917
1000,2448917


In [86]:
ej.tail(4)

,j1
Eddy ID,
997,2448966
998,2448966
999,2448966
1000,2448966


In [87]:
sd = sj.j1.apply(jday_to_datetime)
ed = ej.j1.apply(jday_to_datetime)

In [88]:
sd = pd.DataFrame(sd)
ed = pd.DataFrame(ed)

In [89]:
# Start Dates
sd = sd.rename(columns = {'j1':'date'})

# End Dates
ed = ed.rename(columns = {'j1':'date'})

In [90]:
sd.tail(4)

,date
Eddy ID,
997,1992-10-21 12:00:00
998,1992-10-21 12:00:00
999,1992-10-21 12:00:00
1000,1992-10-21 12:00:00


In [91]:
ed.tail(4)

,date
Eddy ID,
997,1992-12-09 12:00:00
998,1992-12-09 12:00:00
999,1992-12-09 12:00:00
1000,1992-12-09 12:00:00


In [92]:
# Durations
dd = ed-sd

In [93]:
dd.tail(4)

,date
Eddy ID,
997,49 days
998,49 days
999,49 days
1000,49 days


In [94]:
di = np.zeros(M)

In [95]:
for i in range(0, M):
    di[i] = int(dd.date[i+1].astype('timedelta64[D]')/np.timedelta64(1, 'D'))

In [96]:
di = pd.DataFrame(di)

In [97]:
di = pd.concat([index, di], axis=1)
di.columns = ['Eddy ID','day']
di = di.set_index('Eddy ID')
di.day = di.day.astype(int)

In [98]:
di.tail(4)

,day
Eddy ID,
997,49
998,49
999,49
1000,49


### Area ###

In [99]:
radius = np.asarray(ds.L)

In [100]:
area = 2*pi*(radius**2)

In [101]:
arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['Eddy ID', 'Point ID'])

In [102]:
area = pd.DataFrame(area, index=multi_index)
area.columns = ['area']

In [103]:
area.tail(4)

area
Eddy ID Point ID              
1000    5         27548.744141
        6         29656.417969
        7         17034.654297
        8         21294.652344

In [104]:
aa = np.zeros(M)

In [105]:
for i in range(0, M):
    aa[i] = area.loc[i+1].loc[1]

In [106]:
aa = pd.DataFrame(aa)

In [107]:
# Area in Square Kilometers
aa = pd.concat([index, aa], axis=1)
aa.columns = ['Eddy ID','area']
aa = aa.set_index('Eddy ID')

# Area in Square Meters
aa = aa*10**6

In [108]:
aa.tail(4)

,area
Eddy ID,
997,1.003433e+11
998,2.530403e+10
999,4.702965e+10
1000,1.154321e+10


### Vorticity ###

In [109]:
zeta = np.asarray((ds.U*0.01)/(ds.L*1000))

In [110]:
arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['Eddy ID', 'Point ID'])

In [111]:
zeta = pd.DataFrame(zeta, index=multi_index)
zeta.columns = ['relative vorticity']

In [112]:
zeta.tail(4)

relative vorticity
Eddy ID Point ID                    
1000    5               7.722806e-07
        6               8.691537e-07
        7               1.078850e-06
        8               8.651360e-07

In [113]:
lav = np.zeros(M)

In [114]:
for i in range(0, M):
    lav[i] = zeta.loc[i+1].mean()

In [115]:
lav = pd.DataFrame(lav)

In [116]:
lav = pd.concat([index, lav], axis=1)
lav.columns = ['Eddy ID','zeta']
lav = lav.set_index('Eddy ID')

In [117]:
lav.tail(4)

,zeta
Eddy ID,
997,1.072882e-06
998,1.126555e-06
999,1.185512e-06
1000,8.497072e-07


### GeoJSON ###

In [118]:
from geojson import Feature, FeatureCollection, LineString, Point, Polygon
from pymongo import MongoClient
from tqdm import tqdm

In [119]:
client = MongoClient()
db = client.ocean

In [ ]:
# GeoJSON Schema For One Eddy

a = {
    '_id': eddy_id,
    'type': 'FeatureSet',
    'loc_start': [·,·],
    'loc_end': [·,·],
    'date_start': date_start,
    'date_end': date_end,
    'duration': ndays,
    'area': area,
    'lav': lav,
    'features': [
        {
            # Starting Centers
            'type': 'Feature',
            'properties': {'name': 'start_center'},
            'geometry': {'type': 'Point', 'coordinates': [·,·]}
        },
        {
            # Ending Centers
            'type': 'Feature',
            'properties': {'name': 'end_center'},
            'geometry': {'type': 'Point', 'coordinates': [·,·]}
        },
        {
            # Trajectories
            'type': 'Feature',
            'properties': {'name': 'trajectories',},
            'geometry': {'type': 'LineString', 'coordinates': [[·,·],[·,·],[·,·],...]}
        },
        {
            # Origin Polygons
            'type': 'Feature',
            'properties': {'name': 'start_polygon'},
            'geometry': {'type': 'Polygon', 'coordinates': [[[·,·],[·,·],[·,·],...]]}
        },
        {
            # Termination Polygons
            'type': 'Feature',
            'properties': {'name': 'end_polygon'},
            'geometry': {'type': 'Polygon', 'coordinates': [[[·,·],[·,·],[·,·],...]]}
        }
    ]
}

In [121]:
for i in range(1, 2):

    # GeoJSON
    eddy = {
        '_id': int(eddy_id.loc[i]),
        'type': 'FeatureSet',
        'loc_start': [sc.loc[i]['lon'], sc.loc[i]['lat']],
        'loc_end': [ec.loc[i]['lon'], ec.loc[i]['lat']],
        'date_start': sd.loc[i]['date'],
        'date_end': ed.loc[i]['date'],
        'duration': di.loc[i]['day'],
        'area': aa.loc[i]['area'],
        'lav': lav.loc[i]['zeta'],
        'features': [
            {
                'type': 'Feature',
                'properties': {'name': 'start_center'},
                'geometry': Point(tuple(sc.loc[i][['lon', 'lat']].values))
            },
            {
                'type': 'Feature',
                'properties': {'name': 'end_center'},
                'geometry': Point(tuple(ec.loc[i][['lon', 'lat']].values))
            },
            {
                'type': 'Feature',
                'properties': {'name': 'trajectory'},
                'geometry': LineString([tuple(x) for x in ls.loc[i][['lon', 'lat']].values])
            },
            {
                'type': 'Feature',
                'properties': {'name': 'start_polygon'},
                'geometry': Polygon([[tuple(x) for x in op.loc[i][['lon', 'lat']].values]])
            },
            {
                'type': 'Feature',
                'properties': {'name': 'end_polygon'},
                'geometry': Polygon([[tuple(x) for x in tp.loc[i][['lon', 'lat']].values]])
            }
        ]    
    }
    
    # MongoDB
    # result = db.ocean.insert_one(eddy)    

In [124]:
eddy

{'_id': 1,
 'area': 12422155273.4375,
 'date_end': Timestamp('1992-11-04 12:00:00'),
 'date_start': Timestamp('1992-10-14 12:00:00'),
 'duration': 21,
 'features': [{'geometry': {"coordinates": [-83.260009765625, -65.0010986328125], "type": "Point"},
   'properties': {'name': 'start_center'},
   'type': 'Feature'},
  {'geometry': {"coordinates": [-84.155029296875, -65.293296813964844], "type": "Point"},
   'properties': {'name': 'end_center'},
   'type': 'Feature'},
  {'geometry': {"coordinates": [[-83.260009765625, -65.0010986328125], [-84.010009765625, -65.137603759765625], [-84.17999267578125, -65.245697021484375], [-84.155029296875, -65.293296813964844]], "type": "LineString"},
   'properties': {'name': 'trajectory'},
   'type': 'Feature'},
  {'geometry': {"coordinates": [[[-82.31378750048529, -65.0010986328125], [-82.385814381655962, -64.848073338235167], [-82.59092958543502, -64.718344757579487], [-82.897906181421064, -64.631662891257761], [-83.260009765625, -64.601224267644412],

In [125]:
dict(eddy)

{'_id': 1,
 'area': 12422155273.4375,
 'date_end': Timestamp('1992-11-04 12:00:00'),
 'date_start': Timestamp('1992-10-14 12:00:00'),
 'duration': 21,
 'features': [{'geometry': {"coordinates": [-83.260009765625, -65.0010986328125], "type": "Point"},
   'properties': {'name': 'start_center'},
   'type': 'Feature'},
  {'geometry': {"coordinates": [-84.155029296875, -65.293296813964844], "type": "Point"},
   'properties': {'name': 'end_center'},
   'type': 'Feature'},
  {'geometry': {"coordinates": [[-83.260009765625, -65.0010986328125], [-84.010009765625, -65.137603759765625], [-84.17999267578125, -65.245697021484375], [-84.155029296875, -65.293296813964844]], "type": "LineString"},
   'properties': {'name': 'trajectory'},
   'type': 'Feature'},
  {'geometry': {"coordinates": [[[-82.31378750048529, -65.0010986328125], [-82.385814381655962, -64.848073338235167], [-82.59092958543502, -64.718344757579487], [-82.897906181421064, -64.631662891257761], [-83.260009765625, -64.601224267644412],

In [268]:
# Old GeoJSON Codes

for i in range(1, 2):
    
    # geometry
    tuples_sc = tuple(sc.loc[i][['lon', 'lat']].values)
    sc_geo = Point(tuples_sc)
    tuples_ec = tuple(ec.loc[i][['lon', 'lat']].values)
    ec_geo = Point(tuples_ec)
    tuples_ls = [tuple(x) for x in ls.loc[i][['lon', 'lat']].values]
    ls_geo = LineString(tuples_ls)
    tuples_op = [[tuple(x) for x in op.loc[i][['lon', 'lat']].values]]
    op_geo = Polygon(tuples_op)
    tuples_tp = [[tuple(x) for x in tp.loc[i][['lon', 'lat']].values]]
    tp_geo = Polygon(tuples_tp)
   
    # Feature
    sc_f = Feature(geometry=sc_geo, id=i)
    ec_f = Feature(geometry=ec_geo, id=i)
    ls_f = Feature(geometry=ls_geo, id=i)
    op_f = Feature(geometry=op_geo, id=i)
    tp_f = Feature(geometry=tp_geo, id=i)
      
    # FeatureCollection
    fc = FeatureCollection([sc_f, ec_f, ls_f, op_f, tp_f])
    
    # MongoDB
    # result = db.ocean.insert_one(dict(fc))

In [ ]:
# Old GeoJSON Codes
for _, group in tqdm(terminations.groupby(level=0)):
    feature = {"type": "Feature", "geometry": {"type": "Polygon", "coordinates": None}}
    feature["geometry"]["coordinates"] = group.to_json(orient='values')
    result = db.po.insert_one(feature)

In [ ]:
# Old GeoJSON Codes
for _, group in tqdm(terminations.groupby(level=0)):
    feature = {"type": "Feature", "geometry": {"type": "Polygon", "coordinates": None}}
    feature["geometry"]["coordinates"] = group.to_json(orient='values')
    result = db.pt.insert_one(feature)

In [78]:
# Old GeoJSON Codes
for _, group in tqdm(trajectories.groupby(level=0)):
    feature = {"type": "Feature", "geometry": {"type": "LineString", "coordinates": None}}
    feature["geometry"]["coordinates"] = group.to_json(orient='values')
    result = db.ls.insert_one(feature)

In [ ]:
# Print Database Names
client.database_names()

In [ ]:
# Print Collection Names
db.collection_names(client)